In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy.matlib
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from livelossplot import PlotLossesKerasTF
from sklearn.metrics import f1_score, accuracy_score, roc_curve, auc
from sklearn.metrics import mean_absolute_error as mae
from tqdm import tqdm

In [2]:
from torch_geometric.datasets import Planetoid
import torch
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from torch_geometric.data import Dataset, Data, DataLoader
import torch_geometric.nn as gnn
from torch_geometric.utils import softmax
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
from IPython.display import Javascript

In [25]:
inv = 0
# for i in range(2):
#     inv = invs[i]
#     display(Javascript('IPython.notebook.execute_cells_below()'))

In [26]:
df = pd.read_csv('input/CMS_trigger.csv')
print(df.columns)
df

Index(['Unnamed: 0', 'Phi_0', 'Phi_2', 'Phi_3', 'Phi_4', 'Theta_0', 'Theta_2',
       'Theta_3', 'Theta_4', 'BendingAngle_0', 'BendingAngle_2',
       'BendingAngle_3', 'BendingAngle_4', 'TimeInfo_0', 'TimeInfo_2',
       'TimeInfo_3', 'TimeInfo_4', 'RingNumber_0', 'RingNumber_2',
       'RingNumber_3', 'RingNumber_4', 'Front_0', 'Front_2', 'Front_3',
       'Front_4', 'Mask_0', 'Mask_2', 'Mask_3', 'Mask_4',
       'PatternStraightness', 'Zone', 'MedianTheta', 'q/pt', 'PhiAngle',
       'EtaAngle'],
      dtype='object')


,Unnamed: 0,Phi_0,Phi_2,Phi_3,Phi_4,Theta_0,Theta_2,Theta_3,Theta_4,BendingAngle_0,...,Mask_0,Mask_2,Mask_3,Mask_4,PatternStraightness,Zone,MedianTheta,q/pt,PhiAngle,EtaAngle
0,0,58.066666,55.466667,55.466667,56.000000,11.970,11.684999,11.400,11.400,-13.000000,...,0.0,0.0,0.0,0.0,5.0,3.0,105.0,0.181209,-3.075936,1.722345
1,1,64.583336,66.800000,67.066666,67.200000,6.555,6.840000,6.555,6.840,7.000000,...,0.0,0.0,0.0,0.0,3.0,1.0,124.0,-0.146131,-0.167139,2.012122
2,2,71.150000,67.033330,66.266670,65.466670,2.850,2.565000,2.280,2.280,-16.613783,...,0.0,0.0,0.0,0.0,7.0,0.0,127.0,0.367024,2.431823,2.321646
3,3,34.933334,31.200000,31.200000,31.833334,9.690,8.835000,8.835,9.120,-13.000000,...,0.0,0.0,0.0,0.0,6.0,2.0,60.0,0.267774,-1.343305,1.854506
4,4,68.150000,68.266670,68.300000,68.400000,2.565,2.565000,2.565,2.565,0.000000,...,0.0,0.0,0.0,0.0,4.0,0.0,128.0,-0.019179,-3.134433,2.333772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179351,1179351,30.450000,29.066668,28.933332,28.933332,9.405,9.120000,9.120,9.120,0.000000,...,0.0,0.0,0.0,0.0,5.0,2.0,55.0,0.118520,0.522333,-1.833125
1179352,1179352,39.400000,40.133335,40.266666,40.400000,3.705,3.420000,3.420,3.420,-0.000000,...,0.0,0.0,0.0,0.0,4.0,0.0,75.0,-0.084559,-1.585937,-2.262504
1179353,1179353,65.533330,63.433334,62.933334,63.066666,7.125,6.555000,6.555,6.555,-9.000000,...,0.0,0.0,0.0,0.0,5.0,1.0,120.0,0.173085,1.175915,-1.988468
1179354,1179354,41.283333,40.533333,40.366665,40.400000,3.990,3.705000,3.705,3.705,-5.537928,...,0.0,0.0,0.0,0.0,4.0,0.0,77.0,0.085662,2.748315,-2.236174


In [27]:
scaler_1 = StandardScaler()
df.loc[:,'Phi_0':'MedianTheta'] = scaler_1.fit_transform(df.loc[:,'Phi_0':'MedianTheta']) # normalize

In [28]:
features = ['Phi_'+str(i) for i in [0,2,3,4]] + ['Theta_'+str(i) for i in [0,2,3,4]] + \
['Front_'+str(i) for i in [0,2,3,4]] + ['BendingAngle_'+str(i) for i in [0,2,3,4]] + \
['TimeInfo_'+str(i) for i in [0,2,3,4]] + ['RingNumber_'+str(i) for i in [0,2,3,4]] + \
['Mask_'+str(i) for i in [0,2,3,4]] #+ ['PatternStraightness'] + ['Zone'] + ['MedianTheta']
edge_index = torch.tensor([(0,1),(1,2),(2,3),(3,2),(2,1),(1,0)], dtype=torch.long).T
edge_index = [(0,1),(1,2),(2,3),(3,2),(2,1),(1,0)]
# edge_index = [(0,1),(1,2),(2,3)]

In [29]:
# x_train, x_test, pT_tr, pT_ts, inv_pT_tr, inv_pT_ts = train_test_split(df[features].to_numpy(), abs(1/df.loc[:,'q/pt']).to_numpy(), 1/abs(1/df.loc[:,'q/pt']).to_numpy(), test_size = 0.2, random_state = 1)
train_mask, test_mask = train_test_split(df['Unnamed: 0'].to_numpy(), test_size = 0.2, random_state = 1)
x_data = df[features].to_numpy()
pT = abs(1/df.loc[:,'q/pt']).to_numpy()
inv_pT = 1/pT
if inv:
    label = inv_pT
    lr = 0.002
else:
    label = pT
    lr = 0.002
num_features = x_data.shape[-1]
print('Data shape: ' + str(x_data.shape))
print(pT.shape)
print('Len train: '+str(len(train_mask))+', Len test: '+str(len(test_mask)))
print('Num. features: '+str(num_features))

Data shape: (1179356, 28)
(1179356,)
Len train: 943484, Len test: 235872
Num. features: 28


In [9]:
def process_data(i):
    
    data = Data(x=torch.tensor(x_data[i].reshape(-1,4).T, dtype=torch.float), y=torch.tensor(label[i], dtype=torch.float),
                edge_index=torch.tensor(edge_index, dtype = torch.long).T)
    return data

In [10]:
class MPL(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(MPL, self).__init__(aggr='add')
        self.mlp1 = torch.nn.Linear(in_channels*2, out_channels)
        self.mlp2 = torch.nn.Linear(in_channels, out_channels)
        self.mlp3 = torch.nn.Linear(2*out_channels, 1)
        self.mlp4 = torch.nn.Linear(2*out_channels, 1)
        self.mlp5 = torch.nn.Linear(in_channels,16)
        self.mlp6 = torch.nn.Linear(out_channels,16)
        self.mlp7 = torch.nn.Linear(16,1)

    def forward(self, x, edge_index):
#         edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        msg = self.propagate(edge_index, x=x)
        x = F.relu(self.mlp2(x))
        w1 = F.sigmoid(self.mlp3(torch.cat([x,msg], dim=1)))
        w2 = F.sigmoid(self.mlp4(torch.cat([x,msg], dim=1)))
        out = w1*msg + w2*x
        
        return out

    def message(self, x_i, x_j, edge_index):
        msg = F.relu(self.mlp1(torch.cat([x_i, x_j-x_i], dim=1)))
        w1 = F.tanh(self.mlp5(x_i))
        w2 = F.tanh(self.mlp6(msg))
        w = self.mlp7(w1*w2)
        w = softmax(w, edge_index[0])
        return msg*w

In [11]:
class MPNN(torch.nn.Module):
    def __init__(self):
        super(MPNN, self).__init__()
        self.conv1 = MPL(7,128 )
        self.conv2 = MPL(128,64)
        self.conv3 = MPL(64,64 )
        self.conv4 = MPL(64,64 )
        self.lin1 = torch.nn.Linear(128, 128)
        self.lin2 = torch.nn.Linear(128, 16)
        self.lin3 = torch.nn.Linear(16, 16)
        self.lin4 = torch.nn.Linear(16, 1)
        self.lin5 = torch.nn.Linear(128, 128)
        self.lin6 = torch.nn.Linear(128, 16)
        self.lin7 = torch.nn.Linear(16, 16)
        self.lin8 = torch.nn.Linear(16, 1)
        self.global_att_pool1 = gnn.GlobalAttention(torch.nn.Sequential(torch.nn.Linear(64, 1)))
        self.global_att_pool2 = gnn.GlobalAttention(torch.nn.Sequential(torch.nn.Linear(64, 1)))
    
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x1 = self.global_att_pool1(x, batch)
        x = F.relu(self.conv3(x, edge_index))
        x = F.relu(self.conv4(x, edge_index))
        x2 = self.global_att_pool2(x, batch)
        x_out = torch.cat([x1, x2], dim=1)
        x = F.relu(self.lin1(x_out))
        x = F.relu(self.lin2(x))
        x = self.lin3(x)
        x = self.lin4(x).squeeze(1)

        return x

In [12]:
class MyDataset(Dataset):
    def __init__(self, indices=list(range(len(df))), transform=None):
        self.transform = transform
        self.indices = indices
    
    def __getitem__(self, idx):
        return process_data(self.indices[idx])
    
    def __len__(self):
        return len(self.indices)

In [71]:
def pTLossTorch(y_pred,y_true):
#     if not inv:
#         y_pred = torch.pow(y_pred,-1)
#         y_true = torch.pow(y_true,-1)
    y_t = (y_true<80).type(torch.FloatTensor)*y_true.type(torch.FloatTensor) + (y_true>=80).type(torch.FloatTensor)*(y_true<250).type(torch.FloatTensor)*y_true.type(torch.FloatTensor)**2.4 + (y_true>=160).type(torch.FloatTensor)*10 
    return torch.mean(y_t.type(torch.FloatTensor)*torch.pow((y_pred-y_true)/y_true,2).type(torch.FloatTensor))/250

In [80]:
def train(prog_bar = True):
    
    train_losses, test_losses = list(), list()
    min_test_loss = float('inf')
    train_loader = DataLoader(MyDataset(indices=train_mask), batch_size=batch_size)
    test_loader = DataLoader(MyDataset(indices=test_mask), batch_size=batch_size)
    
    for epoch in range(epochs):
        train_loss = 0
        test_loss = 0
        if prog_bar:
            pbar = tqdm(train_loader,position=0)
        else:
            pbar = train_loader
            
        # train
        for data in pbar:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data)
            labels = data.y
            loss = pTLossTorch(out, data.y)
            loss.backward()
            optimizer.step()
            if prog_bar:
                pbar.set_description('pTLoss: '+str(loss.cpu().detach().numpy()))
                train_loss += loss.cpu().detach().numpy()/len(train_loader)
                
        # test
        for data in test_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data)
            labels = data.y
            loss = pTLossTorch(out, data.y)
            test_loss += loss.cpu().detach().numpy()/len(test_loader)
        if test_loss<min_test_loss:
            print('Min loss changed from '+str(min_test_loss)+' to '+str(test_loss))
            min_test_loss = test_loss
            torch.save(model.state_dict(), model_name)
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        if epoch > 10 and min(test_losses[-7:])>min_test_loss+1e-9:
            break
        lr_scheduler.step(test_loss)
        print('Epoch: ', str(epoch+1)+'/'+str(epochs),'| Training pTLoss: ', train_loss, '| Testing pTLoss: ', test_loss)
        
        if not prog_bar:
            plt.plot(train_losses, label="Train Loss")
            plt.plot(test_losses, label="Validation Loss")
            plt.xlabel("# Epoch")
            plt.ylabel("Loss")
            plt.legend(loc='upper right')
            plt.show()
    return train_losses, test_losses

In [81]:
%%javascript
var nb = IPython.notebook;
var kernel = IPython.notebook.kernel;
var command = "NOTEBOOK_FULL_PATH = '" + nb.base_url + nb.notebook_path + "'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [82]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
batch_size = 2**14
epochs = 50
model = MPNN().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=1, factor=0.5)

model_name = 'C:/Users/emrek/' + NOTEBOOK_FULL_PATH[:-6] + 'inv_' + str(inv) + '.pth'
train_losses, test_losses = train(prog_bar=True)

  0%|                                                                                           | 0/58 [00:00<?, ?it/s]

Min loss changed from inf to 7.467650318145752
Epoch:  1/50 | Training pTLoss:  9.56685166523374 | Testing pTLoss:  7.467650318145752


  0%|                                                                                           | 0/58 [00:00<?, ?it/s]

Min loss changed from 7.467650318145752 to 5.881567509969076
Epoch:  2/50 | Training pTLoss:  7.350971328801122 | Testing pTLoss:  5.881567509969076


  0%|                                                                                           | 0/58 [00:00<?, ?it/s]

...

Epoch:  48/50 | Training pTLoss:  1.450015222204143 | Testing pTLoss:  1.5896781206130985


  0%|                                                                                           | 0/58 [00:00<?, ?it/s]

Epoch    49: reducing learning rate of group 0 to 3.1250e-05.
Epoch:  49/50 | Training pTLoss:  1.448987876546794 | Testing pTLoss:  1.5896164019902546


pTLoss: 1.3378887: 100%|███████████████████████████████████████████████████████████████| 58/58 [01:06<00:00,  1.14s/it]


Min loss changed from 1.5889342625935872 to 1.5856068054835004
Epoch:  50/50 | Training pTLoss:  1.4456042873448343 | Testing pTLoss:  1.5856068054835004


In [83]:
min(test_losses)

1.5856068054835004

In [84]:
model_name = 'C:/Users/emrek/' + NOTEBOOK_FULL_PATH[:-6] + 'inv_' + str(inv) + '.pth'
batch_size = 512
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loaded_model = MPNN().to(device)
rand_model = MPNN().to(device)
optimizer = torch.optim.Adam(loaded_model.parameters(), lr=0.002, weight_decay=5e-4)
loaded_model.load_state_dict(torch.load(model_name))

<All keys matched successfully>

In [88]:
test_loader = DataLoader(MyDataset(indices=test_mask), batch_size=batch_size)
# test
test_los = 0
preds = []
for data in tqdm(test_loader,position=0):
    data = data.to(device)
    optimizer.zero_grad()
    out = rand_model(data) # change rand_model to seee if it's trained - should not be the same loss
    preds.append(out.cpu().detach())
    labels = data.y
    loss = pTLossTorch(out, data.y)
    test_los += loss.cpu().detach().numpy()/len(test_loader)
print('Test_loss: '+str(test_los))

100%|████████████████████████████████████████████████████████████████████████████████| 461/461 [00:19<00:00, 23.31it/s]

Test_loss: 12.886728798748369


In [86]:
# save to csv
pred_ls = [float(i) for p in preds for i in p]
len(pred_ls)
df_pred = pd.DataFrame(pred_ls)
df_pred.to_csv('C:/Users/emrek/' + NOTEBOOK_FULL_PATH[:-6] + ' inv_' + str(inv)+'.csv')

In [87]:
print(test_los)

1.589500194968991


In [ ]:
display(Javascript('IPython.notebook.execute_all_cells()'))